# Importowanie bibliotek i funkcji

In [2]:
from requests import Request, Session
import json
import pandas as pd
from datetime import datetime
import time
import folium

# Stworzenie tabeli na dane, określenie miast, których mają dotyczyć dane, podanie klucza API

In [3]:
#Tworzenie tabeli na dane
tabela = pd.DataFrame()

#Tworzenie listy miast, dla których będą pobierane dane
miasta = ['Poznań','Toruń','Kraków','Łódź','Legnica','Lublin','Zielona Góra','Warszawa','Olesno',
                   'Białystok','Gdańsk','Katowice','Rzeszów','Kielce','Olsztyn','Szczecin']

#Podanie API KEY
API_KEY = 'b3965b61-6a83-4ae0-a406-b8a6007c6808'

# Pobieranie, czyszczenie i przekształcanie danych

In [4]:
#Pobieranie nazw województw
url = 'https://api.airvisual.com/v2/states?country=Poland&key='+API_KEY
response = Session().get(url)

#Konwersja nazw województw do listy
dane = json.loads(response.text)
lista_wojewodztw = pd.json_normalize(dane['data'])['state'].tolist()

#Lista miast
lista_miast = ['Poznan','Torun','Krakow','Lodz','Legnica','Lublin','Zielona Gora','Warsaw','Olesno',
               'Bialystok','Gdansk','Katowice','Rzeszow','Kielce','Olsztyn','Szczecin']

for x in range(len(lista_miast)):
    url = 'https://api.airvisual.com/v2/city?city='+lista_miast[x]+'&state='+lista_wojewodztw[x]+'&country=Poland&key='+API_KEY
    response = Session().get(url)

    dane = json.loads(response.text)
    tabela = pd.concat([tabela, pd.json_normalize(dane['data'])])

    #Wstrzyamnie pętli w celu nie przekroczenia ilości zapytań do serwera 
    time.sleep(15)  

#Usuwanie zbędnych kolumn
tabela = tabela.drop(tabela.columns[[1,2,3,7,8,9,15,16]], axis=1)

#Lista nowych nazw kolumn
nowe_nazwy_kolumn = ['Miasto', 'Współrzędne geograficzne', 'Czas pomiaru AQI','AQI','Czas pomiaru danych pogodowych',
                     'Temperatura','Ciśnienie','Wilgotność','Prędkość wiatru']

#Zmiana nazw kolumn
tabela = tabela.rename(columns = dict(zip(tabela.columns[[0,1,2,3,4,5,6,7,8]], nowe_nazwy_kolumn)))

#Uporządkowanie indeksów tabeli
tabela.reset_index(inplace=True, drop=True)

#Konwersja daty i godziny
for x in tabela.index:
    tabela.loc[x, 'Czas pomiaru AQI'] = tabela.loc[x]['Czas pomiaru AQI'][0:19].replace('T',' ')
    tabela.loc[x, 'Czas pomiaru danych pogodowych'] = tabela.loc[x]['Czas pomiaru danych pogodowych'][0:19].replace('T',' ')
    
#Zamiana prędkości wiatru z m/s na km/h
for x in tabela.index:
    tabela.loc[x, 'Prędkość wiatru'] = round(tabela.loc[x, 'Prędkość wiatru'] * 3.6, 2)

#Zmienianie nazw miast na polskie
tabela['Miasto'] = miasta

# Wygląd tabeli

In [5]:
tabela

,Miasto,Współrzędne geograficzne,Czas pomiaru AQI,AQI,Czas pomiaru danych pogodowych,Temperatura,Ciśnienie,Wilgotność,Prędkość wiatru
0,Poznań,"[16.9599, 52.3993]",2023-08-24 08:00:00,51,2023-08-24 08:00:00,23,1018,54,3.71
1,Toruń,"[18.6090825, 53.0102007]",2023-08-24 07:00:00,47,2023-08-24 08:00:00,22,1018,62,8.93
2,Kraków,"[19.949188333333332, 50.010574444444444]",2023-08-24 08:00:00,64,2023-08-24 08:00:00,23,1018,65,7.42
3,Łódź,"[19.4509, 51.775411]",2023-08-24 07:00:00,37,2023-08-24 08:00:00,23,1019,58,1.62
4,Legnica,"[16.18, 51.2044]",2023-08-24 08:00:00,32,2023-08-24 08:00:00,23,1019,57,5.29
5,Lublin,"[22.568623, 51.2592]",2023-08-24 07:00:00,56,2023-08-24 08:00:00,24,1017,44,7.42
6,Zielona Góra,"[15.518861, 51.939783]",2023-08-24 07:00:00,38,2023-08-24 08:00:00,22,1019,53,6.01
7,Warszawa,"[20.9605, 52.2816]",2023-08-24 08:00:00,32,2023-08-24 08:00:00,23,1017,59,11.12
8,Olesno,"[18.416878, 50.876983]",2023-08-24 07:00:00,70,2023-08-24 08:00:00,23,1019,49,9.18
9,Białystok,"[23.1563, 53.1268]",2023-08-24 07:00:00,46,2023-08-24 08:00:00,19,1017,72,12.89


# Określenie potrzebnych danych

In [13]:
#Określenie współrzędnych geograficznych
długość_geo = []
szerokość_geo = [] 

for x in tabela.index:
    długość_geo.append(tabela.loc[x, 'Współrzędne geograficzne'][0])
    szerokość_geo.append(tabela.loc[x, 'Współrzędne geograficzne'][1])

#Określenie jednostek
jednostki = {'AQI':'','Temperatura':'℃','Ciśnienie':'hPa','Wilgotność':'%','Prędkość wiatru':'km/h'}

#Określenie danych do wizualizacji
warianty = ['AQI','Temperatura','Ciśnienie','Wilgotność','Prędkość wiatru']



Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

aqi


# Określenie danych do wizualizacji przez użytkownika

In [15]:
#Wybór wariantu przez użytkownika
wariant = input(f"Które z tych danych chcesz zobaczyć na mapie?\n{', '.join(warianty)}\n\n").lower()
if wariant not in [wybor.lower() for wybor in warianty]:
    print("Nie posiadamy tego typu danych")
else:
    if wariant == 'aqi':
        wartości = tabela[wariant.upper()].to_list()
        wariant = wariant.upper()
    else:
        wartości = tabela[wariant.capitalize()].to_list()
        wariant = wariant.capitalize()
        
    #Przypisanie jednostki
    jednostka = jednostki[wariant]

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

temperatura


# Wizualizacja

In [16]:
#Tworzenie mapy
m = folium.Map(location=[51.9194, 19.1451], zoom_start=6)
 
for name, lat, lon, value in zip(miasta, szerokość_geo, długość_geo, wartości):
     folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f'<div style="font-size: 12px; background-color: blue; color: white; padding: 4px; text-align: center;">{value} {jednostka}</div>',
            icon_size=(40, 30)
        )
    ).add_to(m)

#Wyświetlenie mapy
m

# Pozostałe wizualizacje

In [18]:
AQI

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

aqi


In [19]:
Ciśnienie

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

ciśnienie


In [20]:
Wilgotność

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

wilgotność


In [21]:
Prędkość wiatru

Które z tych danych chcesz zobaczyć na mapie?
AQI, Temperatura, Ciśnienie, Wilgotność, Prędkość wiatru

prędkość wiatru
